In [19]:
%load_ext lab_black
import numpy as np
import pandas as pd
import dapgen
from os.path import join

EUR_BFILE = "/u/project/sriram/ukbiobank/data/geno/cal/filter4"
ALL_BFILE = "out/PLINK/all/chr22"
N_TRAIN = 250000
N_VAL = 20000
OUT_DIR = "out/INDIVLIST"

In [2]:
df_eur_fam = dapgen.read_fam(EUR_BFILE + ".fam")
df_all_fam = dapgen.read_fam(ALL_BFILE + ".fam")
assert len(set(df_eur_fam.index) & set(df_all_fam.index)) == len(df_eur_fam), "EUR_BFILE's individuals must be a subset of ALL_BFILE's individual"

read_fam: only the first two columns [FID, IID] are read, using <FID>_<IID> as index
read_fam: only the first two columns [FID, IID] are read, using <FID>_<IID> as index


In [3]:
np.random.seed(1234)
train_index = np.sort(np.random.choice(np.arange(len(df_eur_fam)), size=N_TRAIN, replace=False))
val_test_index = np.setdiff1d(np.arange(len(df_eur_fam)), train_index)
val_index = np.sort(np.random.choice(val_test_index, size=N_VAL, replace=False))
test_index = np.setdiff1d(val_test_index, val_index)

In [4]:
for name, index in zip(['eur_train', 'eur_val', "eur_test"],
                       [train_index, val_index, test_index]):
    df_eur_fam.iloc[index].astype(str).to_csv(join(OUT_DIR, f"{name}.indiv"), sep='\t', index=False, header=False)

# produce a individual list that excludes eur_train, eur_val, eur_test

In [21]:
df_indiv = []

for prefix in ["eur_train", "eur_val", "eur_test"]:
    df_indiv.append(
        pd.read_csv(
            join(OUT_DIR, prefix + ".indiv"),
            sep="\t",
            header=None,
            names=["FID", "IID"],
        )
    )
df_indiv = pd.concat(df_indiv).reset_index(drop=True)

In [22]:
assert np.all(df_indiv.FID == df_indiv.IID)
assert np.all(df_all_indiv.FID == df_all_indiv.IID)
assert len(set(df_indiv.FID) - set(df_all_indiv.FID)) == 0

In [24]:
df_all_indiv = pd.read_csv(
    "out/PLINK/all/chr1.fam",
    usecols=[0, 1],
    delim_whitespace=True,
    header=None,
    names=["FID", "IID"],
)

df_all_indiv[
    df_all_indiv.FID.isin(set(df_all_indiv.FID) - set(df_indiv.FID))
].sort_values("FID").to_csv(
    join(OUT_DIR, "diverse.indiv"), sep="\t", header=False, index=False
)